In [1]:
# Importing required libraries
import datetime
import time
import pymongo
import pandas as pd
import sys

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient
from pymongo import TEXT

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']

In [3]:
# Url input
SITE_URL = 'https://www.elespectador.com'

In [4]:
# Browser connection Geckodriver
DRIVER_PATH = './geckodriver.exe'

In [5]:
#Connection to the page
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

C:\Users\ALFRED~1\AppData\Local\Temp/ipykernel_10324/4016513992.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
# Root or an absolute path
def make_request(browser, relative_path):
    # Making the request and rendering the browser.
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load.
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis.
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
# Creating a connection to MongoDB and database 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-politica']              

In [8]:
# Getting HTML content for news listing page.
news = []
url = '/archivo/politica'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        news.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [9]:
# Addressing
for n in news:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [10]:
# Database
news

[{'title': 'Voto cristiano, ¿un pecado mortal?',
  'relative_path': '/politica/elecciones-colombia-2022/voto-cristiano-un-pecado-mortal/',
  'datetime': '6 abr 2022  - 9:27 p.\xa0m.',
  'author': 'Alfredo Serrano Zabala',
  'summary': 'Las iglesias cristianas no tienen hoy un líder que los represente de verdad, pero su voto, en gran mayoría, va a ser en contra de Petro. Ese voto nadie lo tiene que guiar ni persuadir, llegará solo.',
  'full_text': 'Colombia está cerca de definir cuál será la persona que regirá sus destinos en los próximos cuatro años. Hay incertidumbre si la designación se conocerá en la primera vuelta (programada para el 29 de mayo) o si será necesaria una segunda vuelta (el domingo 19 de junio). Según la reciente encuesta del Centro Nacional de Consultoría para la revista Semana (publicada el pasado 20 de marzo de 2022), en la intención de voto puntea el exalcalde de Bogotá Gustavo Petro, con un 32 % de favorabilidad, seguido por Federico Gutiérrez, con 23 %. Después

In [11]:
# Storing extracted information for further analysis.
collection.insert_many(news)

In [12]:
# Creating a connection to MongoDB to create the environment module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-judicial']

In [13]:
# Getting HTML content for news listing page.

newsJudicial = []
url = '/archivo/judicial'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsJudicial.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [14]:
# Addressing
for n in newsJudicial:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [15]:
# New database
newsJudicial

[{'title': 'Las víctimas de lesiones oculares del Esmad que tuvieron  que salir del país',
  'relative_path': '/judicial/las-victimas-de-lesiones-oculares-del-esmad-que-tuvieron-que-salir-del-pais/',
  'datetime': '6 abr 2022  - 9:00 p.\xa0m.',
  'author': 'José David Escobar',
  'summary': 'Leidy Cadena y Sara Cárdenas perdieron un ojo, respectivamente, durante el paro nacional de 2021. En los últimos meses, recibieron amenazas y ataques, luego de denunciar los hechos ante las autoridades. Hoy se asilan en Europa y denuncian que hay pocos avances de la Fiscalía para determinar quiénes fueron sus agresores.',
  'full_text': 'Reclamar justicia, ser hostigadas y amenazadas. Sentir terror, miedo a perder la vida. Luego vender todas sus pertenencias y salir corriendo del país para buscar asilo en otro. Llorar. Llorar mucho. Así se podrían resumir los últimos meses de Sara Cárdenas y Leidy Cadena, dos de las decenas de víctimas de lesiones oculares que dejó el Esmad durante el paro nacional

In [16]:
# Storing extracted information for further analysis.
collection.insert_many(newsJudicial)

In [17]:
# Creating a connection to MongoDB to create the science module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-economia']

In [18]:
# Getting HTML content for news listing page.

newsEconomia = []
url = '/archivo/economia'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsEconomia.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [19]:
# Addressing
for n in newsEconomia:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [20]:
# New database
newsEconomia

[{'title': '¿Es hora de comprar dólares?',
  'relative_path': '/economia/finanzas-personales/es-hora-de-comprar-dolares/',
  'datetime': '6 abr 2022  - 9:00 p.\xa0m.',
  'author': 'Camilo Vega Barbosa',
  'summary': 'Los colombianos han gozado varias semanas de dólar a la baja, pero analistas advierten que el dólar barato no duraría mucho. Vea las causas y oportunidades de este fenómeno.',
  'full_text': 'El peso está fuerte respecto al dólar. Si bien no estamos en los niveles de 2014 (cuando un dólar valía $2.000), la tasa de cambio ha venido cayendo de forma significativa hasta rozar los $3.700 en los últimos días. Un claro contraste frente a los casi $4.100 que se registraba en enero pasado. Estos períodos en los que la moneda local se aprecia no han sido frecuentes en los últimos ocho años, por lo que no todos los colombianos saben reaccionar al dólar barato y ver sus implicaciones y oportunidades. Entre ellas las opciones de inversión. También te puede interesar: ¿Las pensiones lo

In [21]:
# Storing extracted information for further analysis.
collection.insert_many(newsEconomia)

In [22]:
# Creating a connection to MongoDB to create the technology module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-investigacion']

In [23]:
# Getting HTML content for news listing page.

newsInvestigacion = []
url = '/archivo/investigacion'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsInvestigacion.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [24]:
# Addressing
for n in newsInvestigacion:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [25]:
# New database
newsInvestigacion

[{'title': 'Crisis carcelaria en Colombia: ¿Qué hacer con el Inpec?',
  'relative_path': '/investigacion/crisis-carcelaria-en-colombia-que-hacer-con-el-inpec/',
  'datetime': '4 abr 2022  - 1:48 p.\xa0m.',
  'author': 'Nicolás Achury González',
  'summary': 'Los paseos del empresario Carlos Mattos por Bogotá, la fuga de alias “Matamba”, narcotraficante del Clan del Golfo, y el escape de Claudia Peñaranda, protagonista del caso Invima, reafirman un problema carcelario histórico en el país. El Inpec ha tenido escándalos de corrupción, fugas, violencia y complicidad con criminales. Expertos proponen una reforma estructural del sistema penitenciario, pero enfocándose en la política criminal del Estado.',
  'full_text': 'Desde hace 23 años, el inspector Nelson Barrera ha estado en los centros penitenciarios de Bogotá. En la actualidad, trabaja en la cárcel La Modelo y es el presidente del Sindicato de Empleados Unidos Penitenciarios (SEUP). Barrera asegura que “La corrupción sí es tan fuert

In [26]:
# Storing extracted information for further analysis.
collection.insert_many(newsInvestigacion)

In [27]:
# Creating a connection to MongoDB to create the economy module. 
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador-bogota']

In [28]:
# Getting HTML content for news listing page.

newsBogota = []
url = '/archivo/bogota'

for n in range(1, 2):
    soup = make_request(browser, url + str(n))
    # Finding the section where news are contained.
    layout = soup.find(class_ = 'Layout-flexAds')
    # Getting blocks from layout.
    blocks = layout.find('section').find_all(recursive = False)
    # Finding and concatenating news cards.
    cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
    # Building a list with title and relative path of the news founded.
    for card in cards:
        newsBogota.append({
            'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
            'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
        })

In [29]:
# Addressing
for n in newsBogota:
    # Getting HTML content for news page.
    soup = make_request(browser, n['relative_path'])
    
    # Extracting news metadata.
    n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
    n['author'] = soup.find(class_ = 'ACredit-Author').get_text()
    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
    
    # Extracting and concatenating news full text.
    paragraphs = soup.find_all(class_ = 'font--secondary')
    n['full_text'] = ' '.join([p.get_text() for p in paragraphs])

In [30]:
# New database
newsBogota

[{'title': 'Se reportan enfrenamientos entre indígenas emberá y el ESMAD en el Parque Nacional',
  'relative_path': '/bogota/se-reportan-enfrenamientos-entre-indigenas-embera-y-el-esmad-en-el-parque-nacional-de-bogota/',
  'datetime': '7 abr 2022  - 12:00 a.\xa0m.',
  'author': 'Redacción Bogotá',
  'summary': 'Los hechos se registran a la altura de la Carrera Séptima con Calle 39, en donde se han registrado detonaciones de gases lacrimógenos y enfrentamiento entre la policía antimotines y los indígenas que se encuentra ubicados en el parque. Hasta el momento, la Policía reporta siete uniformados lesionados.',
  'full_text': 'Fuertes enfrentamientos entre miembros de la comunidad indígena emberá y el Escuadrón Móvil Antidisturbios (Esmad) se registran desde las 9 de la noche en el Parque Nacional de Bogotá, ubicado en la Carrera Séptima con Calle 39, en la localidad Teusaquillo. Reportes publicados a través de redes sociales, señalan que el Esmad realizó una intervención en la zona que

In [31]:
# Storing extracted information for further analysis.
collection.insert_many(newsBogota)